# Variational Inference

### Estimation of Transition matrix and Emission parameters for each state

Libraries:

In [1]:
import numpy as np
import torch
import pyro
from pyro import poutine
import pyro.distributions as dist
from pyro.infer import SVI, TraceEnum_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
import pandas as pd
import tqdm
from models.UnivariateHMM import UnivariateHMM
from models.CopulaHMM import CopulaHMM
from utils.CopulaHelpers import *

Global variables:

In [2]:
DATA_DIR="data/"
HIDDEN_STATES = 3
TRAINING_STEPS= 500

Import data:

In [3]:
data = pd.read_csv(f"{DATA_DIR}hulls_df_matchday2_reduced.csv")
data = data.dropna()
# Convert the areas from m^2 to dam^2 for computational reasons
data["HomeHull"]=data["HomeHull"]/100
data["AwayHull"]=data["AwayHull"]/100
data.head()

,Time [s],Period,HomeHull,AwayHull
0,2.0,1.0,5.982854,8.106641
1,4.0,1.0,6.470239,9.445797
2,6.0,1.0,6.687160,10.615702
3,8.0,1.0,6.578269,12.365166
4,10.0,1.0,6.843430,14.701596


## Univariate (separate) models

In [4]:
sequence_X = torch.tensor(data["HomeHull"].values)
sequence_Y = torch.tensor(data["AwayHull"].values)

#### Home Team

In [5]:
pyro.clear_param_store()

In [6]:
# Guide
guide = AutoDelta(poutine.block(UnivariateHMM, expose=["probs_x", "probs_alpha", "probs_beta"]))
# Optimizer
optimizer = Adam({"lr": 0.01})
# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(UnivariateHMM, guide, optimizer, loss=elbo)
# Training
tqdm_bar = tqdm.tqdm(range(TRAINING_STEPS))
for step in tqdm_bar:
    loss = svi.step(sequence_X, HIDDEN_STATES)
    if step % 100 == 0:
         tqdm_bar.set_postfix({'LOSS': loss})

100%|██████████| 500/500 [09:23<00:00,  1.13s/it, LOSS=5.12e+3]


In [7]:
posterior_HomeTeam = guide(sequence_X,HIDDEN_STATES)
posterior_HomeTeam

{'probs_x': tensor([[0.9656, 0.0264, 0.0080],
         [0.8818, 0.0997, 0.0184],
         [0.0807, 0.0360, 0.8834]], grad_fn=<ExpandBackward0>),
 'probs_alpha': tensor([5.6445, 2.9927, 1.3998], grad_fn=<ExpandBackward0>),
 'probs_beta': tensor([0.6066, 0.3261, 0.2830], grad_fn=<ExpandBackward0>)}

In [8]:
torch.save(posterior_HomeTeam,f"parameters/univariateHMM_matchday2_HomeTeam_{HIDDEN_STATES}states.pt")

#### Away Team

In [9]:
pyro.clear_param_store()

In [10]:
# Guide
guide = AutoDelta(poutine.block(UnivariateHMM, expose=["probs_x", "probs_alpha", "probs_beta"]))
# Optimizer
optimizer = Adam({"lr": 0.01})
# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(UnivariateHMM, guide, optimizer, loss=elbo)
# Training
tqdm_bar = tqdm.tqdm(range(TRAINING_STEPS))
for step in tqdm_bar:
    loss = svi.step(sequence_Y, HIDDEN_STATES)
    if step % 100 == 0:
         tqdm_bar.set_postfix({'LOSS': loss})

100%|██████████| 500/500 [09:36<00:00,  1.15s/it, LOSS=5.07e+3]


In [11]:
posterior_AwayTeam = guide(sequence_Y,HIDDEN_STATES)
posterior_AwayTeam

{'probs_x': tensor([[0.1630, 0.4516, 0.3854],
         [0.0065, 0.9811, 0.0123],
         [0.1619, 0.2146, 0.6235]], grad_fn=<ExpandBackward0>),
 'probs_alpha': tensor([2.2528, 9.6323, 1.5598], grad_fn=<ExpandBackward0>),
 'probs_beta': tensor([0.3446, 0.9796, 0.2315], grad_fn=<ExpandBackward0>)}

In [12]:
torch.save(posterior_AwayTeam,f"parameters/univariateHMM_matchday2_AwayTeam_{HIDDEN_STATES}states.pt")

## Bivariate copula model

In [4]:
sequence_XY = torch.tensor(data[["HomeHull","AwayHull"]].values)
sequence_XY.shape

torch.Size([1970, 2])

In [5]:
pyro.clear_param_store()

In [15]:
# Guide
guide = AutoDelta(poutine.block(CopulaHMM, expose=["probs_x",
                                               "probs_alpha1",
                                               "probs_beta1",
                                               "probs_alpha2",
                                               "probs_beta2",
                                               "theta"
                                               ]))
# Optimizer
optimizer = Adam({"lr": 0.01})

# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(CopulaHMM, guide, optimizer, loss=elbo)

# Training
tqdm_bar = tqdm.tqdm(range(TRAINING_STEPS))
for step in tqdm_bar:
    loss = svi.step(sequence_XY, HIDDEN_STATES)
    #if step % 50 == 0:
    tqdm_bar.set_postfix({'LOSS': loss})

100%|██████████| 500/500 [22:37<00:00,  2.72s/it, LOSS=9.28e+3]


In [16]:
posterior = guide(sequence_XY,HIDDEN_STATES)
#posterior

In [17]:
torch.save(posterior,f"parameters/CopulaHMM_matchday2_{HIDDEN_STATES}states.pt")

In [18]:
meanH_state0 = posterior["probs_alpha1"][0]/posterior["probs_beta1"][0]*100
meanA_state0 = posterior["probs_alpha2"][0]/posterior["probs_beta2"][0]*100
meanH_state1 = posterior["probs_alpha1"][1]/posterior["probs_beta1"][1]*100
meanA_state1 = posterior["probs_alpha2"][1]/posterior["probs_beta2"][1]*100
meanH_state2 = posterior["probs_alpha1"][2]/posterior["probs_beta1"][2]*100
meanA_state2 = posterior["probs_alpha2"][2]/posterior["probs_beta2"][2]*100

print(f">> Mean of the Convex Hull for home team (STATE 0): {meanH_state0:.2f} m^2")
print(f">> Mean of the Convex Hull for away team (STATE 0): {meanA_state0:.2f} m^2")
print(f">> Mean of the Convex Hull for home team (STATE 1): {meanH_state1:.2f} m^2")
print(f">> Mean of the Convex Hull for away team (STATE 1): {meanA_state1:.2f} m^2")
print(f">> Mean of the Convex Hull for home team (STATE 2): {meanH_state2:.2f} m^2")
print(f">> Mean of the Convex Hull for away team (STATE 2): {meanA_state2:.2f} m^2")

>> Mean of the Convex Hull for home team (STATE 0): 1001.03 m^2
>> Mean of the Convex Hull for away team (STATE 0): 640.83 m^2
>> Mean of the Convex Hull for home team (STATE 1): 594.12 m^2
>> Mean of the Convex Hull for away team (STATE 1): 1096.94 m^2
>> Mean of the Convex Hull for home team (STATE 2): 1067.54 m^2
>> Mean of the Convex Hull for away team (STATE 2): 1220.47 m^2


# Try with initial probability distribution

In [6]:
import torch
import pyro
import pyro.distributions as dist
from pyro import poutine
from utils.CopulaHelpers import copulamodel_log_pdf

def CopulaHMM(sequence: torch.tensor, 
              hidden_dim: int, 
              include_prior: bool=True):
    '''
    Pyro Model for a Hidden Markov Model with a bivariate observation with Copula emission distribution.
    Structure of the model taken from the Pyro documentation:
    https://pyro.ai/examples/hmm.html
    
    INPUTS:
    - sequence (torch.tensor): A 2-dimensional tensor of observations.
    - hidden_dim (int): The number of hidden states.
    - include_prior (bool): If True, include priors for the parameters of the model.
    '''
    n_obs = sequence.shape[0]
    with poutine.mask(mask=include_prior):
        #---------------------------------------------------------------------
        # Prior for the initial state probabilities
        probs_initial = pyro.sample(
            "probs_initial",
            dist.Dirichlet(torch.ones(hidden_dim))
        )
        #---------------------------------------------------------------------
        # Transition probabilities
        probs_x = pyro.sample(
            "probs_x",
            dist.Dirichlet(0.9 * torch.eye(hidden_dim) + 0.1).to_event(1),
        )
        #---------------------------------------------------------------------
        # Prior for the parameters of emission probabilities 
        probs_alpha1 = pyro.sample(
            "probs_alpha1",
            dist.Gamma(concentration=15.0, rate=0.8).expand([hidden_dim]).to_event(1)
        )

        probs_beta1 = pyro.sample(
            "probs_beta1",
            dist.Gamma(concentration=1.0, rate=1.0).expand([hidden_dim]).to_event(1)
        )
        probs_alpha2 = pyro.sample(
            "probs_alpha2",
            dist.Gamma(concentration=15.0, rate=0.8).expand([hidden_dim]).to_event(1)
        )

        probs_beta2 = pyro.sample(
            "probs_beta2",
            dist.Gamma(concentration=1.0, rate=1.0).expand([hidden_dim]).to_event(1)
        )
        #---------------------------------------------------------------------
        # Prior for theta
        theta = pyro.sample(
            "theta",
            dist.Gamma(5.0, 0.7).expand([hidden_dim]).to_event(1)
        )
        
    # Sample the initial hidden state
    x = pyro.sample(
        "x_0",
        dist.Categorical(probs_initial),
        infer={"enumerate": "parallel"},
    )

    for t in pyro.markov(range(n_obs)):
        if t > 0:
            x = pyro.sample(
                f"x_{t}",
                dist.Categorical(probs_x[x]),
                infer={"enumerate": "parallel"},
            )
        
        log_pdf = copulamodel_log_pdf(
            x=sequence[t,0],
            y=sequence[t,1],
            shape1=probs_alpha1[x],
            rate1=probs_beta1[x],
            shape2=probs_alpha2[x],
            rate2=probs_beta2[x],
            theta=theta[x]
        )
        pyro.factor(f"xy_{t}", log_pdf)

guide = AutoDelta(poutine.block(CopulaHMM, expose=["probs_initial",
                                                  "probs_x",
                                                  "probs_alpha1",
                                                  "probs_beta1",
                                                  "probs_alpha2",
                                                  "probs_beta2",
                                                  "theta"
                                                  ]))

In [7]:
# Optimizer
optimizer = Adam({"lr": 0.01})

# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(CopulaHMM, guide, optimizer, loss=elbo)

# Training
tqdm_bar = tqdm.tqdm(range(TRAINING_STEPS))
for step in tqdm_bar:
    loss = svi.step(sequence_XY, HIDDEN_STATES)
    #if step % 50 == 0:
    tqdm_bar.set_postfix({'LOSS': loss})

 20%|██        | 100/500 [21:21<1:25:26, 12.82s/it, LOSS=1e+4]  


KeyboardInterrupt: 

In [8]:
posterior = guide(sequence_XY,HIDDEN_STATES)
posterior

{'probs_initial': tensor([0.5133, 0.3400, 0.1467], grad_fn=<ExpandBackward0>),
 'probs_x': tensor([[0.7390, 0.1640, 0.0970],
         [0.1859, 0.6482, 0.1659],
         [0.0825, 0.0825, 0.8350]], grad_fn=<ExpandBackward0>),
 'probs_alpha1': tensor([ 9.9712, 12.6953, 17.5064], grad_fn=<ExpandBackward0>),
 'probs_beta1': tensor([1.4381, 1.1009, 0.1918], grad_fn=<ExpandBackward0>),
 'probs_alpha2': tensor([ 9.9901, 10.5457, 17.4985], grad_fn=<ExpandBackward0>),
 'probs_beta2': tensor([0.9273, 1.4040, 0.1743], grad_fn=<ExpandBackward0>),
 'theta': tensor([2.7667, 5.5733, 5.7135], grad_fn=<ExpandBackward0>)}

In [9]:
print(9.9712/1.4381*100)
print(12.6953/1.1009*100)
print(17.5064/0.1918*100)
print(9.9901/0.9273*100)
print(10.5457/1.4040*100)
print(17.4985/0.1743*100)

693.3592935122731
1153.1746752656916
9127.424400417101
1077.3320392537476
751.1182336182337
10039.300057372346
